In [ ]:
import cpm
version = cpm.__version__
if version < "0.18.5":
    raise Exception("cpm version is too old, please update to 0.18.5 or later")
else:
    print(f"cpm version is up to date\nversion: {version}\nauthor: {cpm.__author__}")


In [ ]:
data_pt = pd.read_csv("scavenger_data_python.csv")
## subset data so we can debug
data_pt = data_pt[data_pt["ppt"] == 1]
# Rename columns to match expectations:
# reward_1 = safe_magn, reward_2 = risky_magn, loopID -> ppt, choice -> observed
data_pt.rename(columns={"reward_1": "safe_magn", 
                        "reward_2": "risky_magn", 
                        "loopID": "ppt", 
                        "choice": "observed"}, inplace=True)

print("Data head:")
print(data_pt.head())


In [ ]:
# Order: [alpha, lambda_loss, gamma, temperature]
parameters_settings = [
    [3.0, 0.0, 5.0],   # alpha: starting value 3.0
    [2.0, 0.0, 5.0],   # lambda_loss: starting value 2.0
    [0.5, 0.0, 0.5],   # gamma: starting value 0.5
    [2.0, 0.1, 15.0]   # temperature: starting value 2.0
]


In [ ]:
unique_ppts = data_pt["ppt"].unique()[:2]
print("Testing for participants:", unique_ppts)

# Define a list of weighting functions to test.
weighting_functions = ["tk", "power", "prelec", "gw"]

# Dictionaries to store simulation and optimization results.
simulation_results = {}
optimization_results = {}


In [ ]:
import copy
from cpm.optimisation import minimise, FminBound

for wf in weighting_functions:
    simulation_results[wf] = {}
    optimization_results[wf] = {}
    print("\n==========================")
    print(f"Testing weighting function: {wf}")
    print("==========================")
    
    print(f"\n-- Weighting: {wf}, Participant: {ppt} --")
    ppt_data = data_pt[data_pt["ppt"] == 1].copy()
    # Instantiate the prospect theory model for simulation
    import Prospect_theory_wrapper_ben
    pt_model = Prospect_theory_wrapper_ben.ProspectSoftmaxModel(
        data=ppt_data,
        parameters_settings=parameters_settings,
        generate=False,
        mode="safe_risky",
        weighting=wf
    )
    pt_model.run()
    output = pt_model.export()

    print(f"Simulation output (first 10 rows) for weighting '{wf}':")
    print(output.head(10))
    
    # Now, for optimization, make a deep copy of the model instance.
    model_for_opt = copy.deepcopy(pt_model)
    grouped_data = data_pt.groupby("ppt")
    
    fit = FminBound(
        model=model_for_opt, 
        data=grouped_data,
        minimisation=minimise.LogLikelihood.bernoulli,
        parallel=True,
        libraries=["numpy", "cpm", "pandas"],
        cl=5,
        prior=False,
        ppt_identifier="ppt",
        display=False,
        number_of_starts=5,
        approx_grad=True
    )
    
    try:
        fit.optimise()
        opt_result = fit.export()
    except Exception as e:
        print(f"Error during optimization for weighting {wf}")
        opt_result = None

    optimization_results[wf] = opt_result


In [ ]:

#this one here does not work.
from cpm.generators import Simulator

# If your optimization was run on data grouped by 'ppt' for all participants:
simulator = Simulator(
    wrapper=pt_model,         # your prospect theory model instance
    parameters=fit.parameters,  # parameter sets for all participants
    data= grouped_data  # group by 'ppt' so that the number of groups matches
)